In [2]:
# Created by Hyunjun,JANG
# limitsinx.tistory.com
# Last revision date : 2021.08.03

import numpy as np
import pandas as pd
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import datetime
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')

# GPU setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#print(torch.cuda.get_device_name(0)) #Google colab = TESLA T4

# Training/Test ratio = 7:3
Train_ratio = 0.7
Test_ratio = 0.3

# Data load
df = pd.read_csv('./LSTM_데이터/housing.csv')
x = df.iloc[:,0:-1]
y = df.iloc[:,-1:]

# Data Division
train_x = x.iloc[0:int(len(df)*Train_ratio),:]
train_y = y.iloc[0:int(len(df)*Train_ratio),:]
test_x = x.iloc[int(len(df)*Train_ratio):,:]
test_y = y.iloc[int(len(df)*Train_ratio):,:]

# Normalizing , 둘다 학습하여 성능비교할것
minmax = MinMaxScaler()
standard = StandardScaler()

train_x = minmax.fit_transform(train_x)
train_y = minmax.fit_transform(train_y)
test_x = minmax.fit_transform(test_x)
test_y = minmax.fit_transform(test_y)

# Check Data pre-processing
#print("Training shape : ", train_x.shape, train_y.shape)
#print("Test shape : ",test_x.shape, test_y.shape)

# Numpy array상태로는 학습이 불가능하므로, Torch Variable 형태로 변경(data/grad/grad_fn)
train_x_tensor = Variable(torch.Tensor(train_x))
train_y_tensor = Variable(torch.Tensor(train_y))
#print("After torch variable shape_Train : ",train_x_tensor.shape, train_y.shape)

test_x_tensor = Variable(torch.Tensor(test_x))
test_y_tensor = Variable(torch.Tensor(test_y))
#print("After torch Variable shape_Test : ",test_x_tensor.shape, test_y_tensor.shape)

train_x_tensor_final = torch.reshape(train_x_tensor, (train_x_tensor.shape[0], 1, train_x_tensor.shape[1]))
train_y_tensor_final = torch.reshape(train_y_tensor, (train_y_tensor.shape[0], 1, train_y_tensor.shape[1]))
test_x_tensor_final = torch.reshape(test_x_tensor, (test_x_tensor.shape[0], 1, test_x_tensor.shape[1]))
test_y_tensor_final = torch.reshape(test_y_tensor,(test_y_tensor.shape[0], 1, test_y_tensor.shape[1]) )
#print(train_x_tensor_final.shape, test_x_tensor_final.shape)

# LSTM network modeling
class LSTM_Jun(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length) :
        super(LSTM_Jun, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, batch_first = True)
        self.layer_1 = nn.Linear(hidden_size, 256)
        self.layer_2 = nn.Linear(256,256)
        self.layer_3 = nn.Linear(256,128)
        self.layer_out = nn.Linear(128, num_classes)
        self.relu = nn.ReLU() #Activation Func

    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #Hidden State
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #Internal Process States

        output, (hn, cn) = self.lstm(x, (h_0, c_0))

        hn = hn.view(-1, self.hidden_size) # Reshaping the data for starting LSTM network
        out = self.relu(hn) #pre-processing for first layer
        out = self.layer_1(out) # first layer
        out = self.relu(out) # activation func relu
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.layer_3(out)
        out = self.relu(out)
        out = self.layer_out(out) #Output layer
        return out

# Code Main
num_epochs = 10000
learning_rate = 0.001
input_size = int(len(x.columns))
hidden_size = 2 # number of features in hidden state
num_layers = 1
num_classes = int(len(y.columns))

LSTM_Jun = LSTM_Jun(num_classes, input_size, hidden_size, num_layers, train_x_tensor_final.shape[1]).to(device)

loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(LSTM_Jun.parameters(), lr = learning_rate)

for epoch in range(num_epochs) :
    outputs = LSTM_Jun.forward(train_x_tensor_final.to(device))
    optimizer.zero_grad()
    loss = loss_function(outputs, train_y_tensor.to(device))
    loss.backward()
    optimizer.step() # improve from loss = back propagation
    if epoch % 200 == 0 :
        print("Epoch : %d, loss : %1.5f" % (epoch, loss.item()))

# Estimated Value
test_predict = LSTM_Jun(train_x_tensor_final.to(device)) #Forward Pass
predict_data = test_predict.data.detach().cpu().numpy() #numpy conversion
predict_data = minmax.inverse_transform(predict_data) #inverse normalization(Min/Max)

# Real Value
real_data = train_y_tensor.data.numpy() # Real value
real_data = minmax.inverse_transform(real_data) #inverse normalization

#Figure
plt.figure(figsize = (10,6)) # Plotting
plt.plot(real_data, label = 'Real Data')
plt.plot(predict_data, label = 'predicted data')
plt.title('Time series prediction')
plt.legend()
plt.show()



ValueError: could not convert string to float: 'NEAR BAY'